# **ก่อนเริ่มต้นใช้งาน**

ที่แถบเมนูด้านบนคลิกที่ File และเลือก Save a copy in Drive เพื่อเซฟเป็นของตัวเอง

# **Setup Colab Notebook**

การเตรียมพร้อมสำหรับใช้ Colab มีดังนี้


1.   คลิกที่ Runtime เลือก Change runtime type และที่ Hardware accelerator เลือก GPU
2.   Mounted Google Drive เพื่อเชื่อมต่อกับไดร์ฟของเราโดยจะอนุญาตให้ Colab สามารถ อ่าน เขียน ลบ แก้ไข ไฟล์ต่าง ๆ ในไดร์ฟของเราได้
3. เตรียม Directory สำหรับทำงาน ซึ่งสามารถทำได้ 2 วิธีคือใช้ Command (Colab ใช้ Linux) หรือจะจัดการผ่าน UI ที่แถบด้านข้างหรือผ่านไดร์ฟก็ได้
<br /> หมายเหตุ หากจะใช้ Command จะต้องใช้เครื่องหมาย ! หรือ % นำหน้า




In [1]:
# เข้าถึง Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#สร้างโฟลเดอร์ชื่อ AIFORTHAI
%mkdir AIFORTHAI

In [8]:
# Change Directory ไปที่ Google Drive
%cd drive/MyDrive/AIFORTHAI

[Errno 2] No such file or directory: 'drive/MyDrive/AIFORTHAI'
/content/drive/MyDrive/AIFORTHAI/finetune-wav2vec2-ISANDialect


In [4]:
# สร้างโฟลเดอร์ชื่อ finetune-wav2vec2-ISANDialect
%mkdir finetune-wav2vec2-ISANDialect

In [5]:
# เข้าสู่โฟลเดอร์ที่สร้างข้างต้น
%cd finetune-wav2vec2-ISANDialect

/content/drive/MyDrive/AIFORTHAI/finetune-wav2vec2-ISANDialect


# **Install Library**

Library ต่าง ๆ ที่จำเป็นกับงานสามารถติดตั้งได้โดยใช้ pip install ตามด้วยชื่อไลบรารี่ <br>pip คือ Preferred Installer Program หรือก็คือ Package Installer สำหรับ Python <br /> ข้อควรรู้คือเราจะต้องติดตั้งไลบรารี่ใหม่ทุกครั้งที่ Runtime หยุดทำงานและถูกลบไป (Disconnect & Delete Runtime)

*   transformers ใช้สำหรับโหลด Model, Tokenizer, Feature Extractor จาก Hub ของ Hugging Face
*   datasets ใช้โหลดชุดข้อมูลจาก Hub ของ Hugging Face
*   librosa สำหรับเปิดไฟล์เสียง
*   jiwer และ evaluate สำหรับวัดผลการทำงานของโมเดล ใช้ประเมินผลตัวโมเดลด้วย WER
*   lextoplus และ PyThaiNLP สำหรับตัดคำในประโยคภาษาไทย





In [6]:
# Need to install every time after terminate runtine
!pip install transformers
!pip install datasets
!pip install librosa
!pip install jiwer
!pip install evaluate
!pip install pythainlp
!pip install lextoplus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


# **Prepare Data ISAN Dialect**

เตรียมชุดข้อมูลเสียงสำเนียงถิ่นชุดเล็กเพื่อการทดสอบ ประกอบด้วย
* ชุดฝึกสอน (Train Set) 10 ผู้พูด  1000 ประโยค
* ชุดปรับจูน (Validation Set) 1 ผู้พูด 100 ประโยค
* ชุดทดสอบ (Test Set) 1 ผู้พูด 100 ประโยค

ในกรณที่ใช้ข้อมูลชุดอื่น ก็ควรจะต้องเตรียมให้อยู่ในฟอร์แมตนี้เช่นกัน <br />
<a href="https://colab.research.google.com/drive/1Ht0kkW5JZ-96J9sekxh6d9HWZbEG0NL9?usp=sharing"> ดังเช่นวิธีการเตรียมชุดข้อมูลจากในตัวอย่างนี้
โดยปกติชุดข้อมูลที่โหลดมาจะถูกเก็บใน .cache ของเครื่อง <br />

คอลัมน์หลัก ๆ ที่ต้องการใช้งานคือ
*   audio โดยสนใจ Array ของ Wav และ Sample Rate
*   sentence สำหรับใช้เป็น label

 </a>



In [ ]:
# โหลดข้อมูลจาก HuggingFace Hub
from datasets import load_dataset

train_ds = load_dataset("common_voice", "th", split="train")
validation_ds = load_dataset("common_voice", "th", split="validation")
test_ds = load_dataset("common_voice", "th", split="test")

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/2917 [00:00<?, ? examples/s]

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating test split:   0%|          | 0/2188 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1922 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/2671 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/7028 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/467 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/th/6.1.0/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf. Subsequent calls will reuse this data.


In [ ]:
# จำนวนแถวและคอลัมน์ของชุดข้อมูล
print("Train Datasets:", train_ds.shape)
print("Validation Datasets:", validation_ds.shape)
print("Test Datasets:", test_ds.shape)

Train Datasets: (2917, 11)
Validation Datasets: (1922, 11)
Test Datasets: (2188, 11)


In [ ]:
# ลบคอลัมน์ที่ไม่จำเป็นต้องใช้งาน
train_ds = train_ds.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
validation_ds = validation_ds.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
test_ds = test_ds.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
# เรียกดูข้อมูล ทำได้โดยใส่ datasets[index] จะเรียกดู 1 แถวตาม index ที่ระบุ
# หากต้องการดูหลายแถวพร้อมกันทำได้โดย datasets[index, index, index]
train_ds[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23654854.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23654854.mp3',
  'array': array([ 0.0000000e+00, -1.1551621e-13,  2.5164576e-14, ...,
         -2.6744937e-07,  9.9478768e-07, -1.3387196e-07], dtype=float32),
  'sampling_rate': 48000},
 'sentence': 'เงียบหน่อย เจ้าหนู'}

In [ ]:
# เรียกดู 2 แถวพร้อมกัน
validation_ds[0, 1]

{'path': ['/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23646618.mp3',
  '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23646621.mp3'],
 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23646618.mp3',
   'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 6.9911789e-06,
          4.5574448e-06, 2.4187045e-06], dtype=float32),
   'sampling_rate': 48000},
  {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23646621.mp3',
   'array': array([ 0.00000000e+00,  1.22805537

In [ ]:
# เรียกดู 3 แถวพร้อมกัน
test_ds[0, 1, 2]

{'path': ['/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23654342.mp3',
  '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23654343.mp3',
  '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23654345.mp3'],
 'audio': [{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3fa1d036f7dd806dce529a685d419d747ac2330f88abf8f1009d6b0ab683e8e7/cv-corpus-6.1-2020-12-11/th/clips/common_voice_th_23654342.mp3',
   'array': array([ 0.0000000e+00,  9.6685169e-15,  1.4178625e-14, ...,
           9.8782175e-07,  7.3779506e-07, -1.1138891e-06], dtype=float32),
   'sampling_rate': 48000},
  {'path': '/root/.cache/huggingface/datasets/down

# **Data Cleaning**

จัดการข้อความประโยคภาษาไทยโดยจะลบสัญลักษณ์พิเศษและตัวเลขออก <br />
เพื่อนำประโยคส่วนนี้ไปใช้สร้าง Tokenizer และใช้เป็น Label เฉลยตอนวัดผลโมเดล





In [ ]:
# ฟังก์ชันสำหรับ Remove อักขระทุกตัวยกเว้นภาษาไทยและภาษาอังกฤษ โดยใช้ Regular Expression
import re
chars = r"[^\u0E00-\u0E7Fa-zA-Z ]+"

def remove_chars(batch):
  text = batch["sentence"]
  batch["cleanSentence"] = re.sub(chars, "", text)

  return batch

In [ ]:
# จัดการประโยคในชุด train
train_ds = train_ds.map(remove_chars, remove_columns=["sentence"], num_proc=1)

Map:   0%|          | 0/2917 [00:00<?, ? examples/s]

In [ ]:
# จัดการประโยคในชุด validation
validation_ds = validation_ds.map(remove_chars, remove_columns=["sentence"], num_proc=1)

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

In [ ]:
# จัดการประโยคในชุด test
test_ds = test_ds.map(remove_chars, remove_columns=["sentence"], num_proc=1)

Map:   0%|          | 0/2188 [00:00<?, ? examples/s]

# **Create Tokenizer and Feature Extractor**

มีขั้นตอนดังนี้

1.   สร้างลิสต์ของอักขระทั้งหมดที่พบในชุดข้อมูล
2.   สร้าง Dictionary โดยมี Key คือ อักขระ และ Value คือ Index ของอักขระนั้น
3. สำหรับวรรค " " ใช้สัญลักษณ์ | แทนที่
4. เพิ่ม UNK สำหรับคำที่ไม่รู้จักหรือไม่เคยเห็นมาก่อน
5. เพิ่ม PAD สำหรับตำแหน่งที่ถูกเพิ่มเพื่อให้ลิสต์ของโทเค็นมีขนาดเท่ากัน
6. บันทึก Dictionary ที่สร้างเสร็จแล้วเป็น JSON เพื่อเก็บไว้ใช้ซ้ำ
7. สร้าง Tokenizer จากไฟล์ Dictionary ที่บันทึกไว้
8. สร้าง Feature Extractor สำหรับจัดการข้อมูลเสียง
9. สร้าง Processor โดยแพ็ค Tokenizer และ Feature Extractor ไว้เพื่อให้ง่ายต่อการเรียกใช้งาน



In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["cleanSentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train_ds = train_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_ds.column_names)
vocab_validation_ds = validation_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=validation_ds.column_names)
vocab_test_ds = test_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_ds.column_names)

Map:   0%|          | 0/2917 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/2188 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train_ds["vocab"][0]) | set(vocab_validation_ds["vocab"][0]) | set(vocab_test_ds["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ฑ': 0,
 'โ': 1,
 'ฤ': 2,
 'ข': 3,
 'ญ': 4,
 'ต': 5,
 'ช': 6,
 'ิ': 7,
 'ๅ': 8,
 ' ': 9,
 'ด': 10,
 'ฬ': 11,
 'ป': 12,
 'ฒ': 13,
 '่': 14,
 'ท': 15,
 'ฏ': 16,
 'ื': 17,
 '็': 18,
 'ฆ': 19,
 'ฮ': 20,
 'ฝ': 21,
 'เ': 22,
 'ศ': 23,
 'ก': 24,
 'ฉ': 25,
 'ฐ': 26,
 'ไ': 27,
 'ร': 28,
 'ษ': 29,
 'ํ': 30,
 'ถ': 31,
 'แ': 32,
 '๋': 33,
 '์': 34,
 'ู': 35,
 'ภ': 36,
 'ณ': 37,
 'ซ': 38,
 'ั': 39,
 'บ': 40,
 'ง': 41,
 'ฎ': 42,
 'ๆ': 43,
 'ม': 44,
 'ย': 45,
 'ใ': 46,
 'อ': 47,
 'ะ': 48,
 '้': 49,
 'พ': 50,
 'ผ': 51,
 'น': 52,
 'ล': 53,
 'ว': 54,
 'จ': 55,
 'ึ': 56,
 'ี': 57,
 'ค': 58,
 'ฟ': 59,
 'ส': 60,
 'ำ': 61,
 'ุ': 62,
 'ธ': 63,
 'า': 64,
 '๊': 65,
 'ห': 66}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

69

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True, return_attention_mask=True)

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("wav2vec2-xlsr53-TH-cmv-processor")

# **Prepared Data**

1. แปลง Sample Rate ของไฟล์เสียงเป็น 16kHz
2. สร้างข้อมูล Input จากไฟล์เสียง สำหรับฝึกสอนโมเดลด้วย Feature Extractor
3. ตัดคำภาษาไทยเพื่อให้สามารถวัดผลด้วย WER ได้
4. สร้างข้อมูล Input จากข้อความ สำหรับใช้เป็นเฉลยในการฝึกสอนโมเดลด้วย Tokenizer
5. บันทึกชุดข้อมูลที่เตรียมเสร็จสิ้นแล้วเพื่อเก็บไว้ใช้ซ้ำ



In [ ]:
from datasets import Audio

train_ds = train_ds.cast_column("audio", Audio(sampling_rate=16_000))
validation_ds = validation_ds.cast_column("audio", Audio(sampling_rate=16_000))
test_ds = test_ds.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
import IPython.display as ipd
import numpy as np
import random

# ทดสอบฟังเสียง
print("Target text:", train_ds[500]["cleanSentence"])
print("Input array shape:", train_ds[500]["audio"]["array"].shape)
print("Sampling rate:", train_ds[500]["audio"]["sampling_rate"])

ipd.Audio(data=train_ds[500]["audio"]["array"], autoplay=True, rate=train_ds[500]["audio"]["sampling_rate"])

Target text: มันเป็นเรื่องที่ยากที่จะค้นหาที่อยู่จากข้อมูลและคำอธิบายอันคลุมเครือที่พวกเรามี
Input array shape: (106752,)
Sampling rate: 16000


In [ ]:
from pythainlp.tokenize import word_tokenize

def th_words_tokenize(batch):
  sentence = batch["cleanSentence"]
  tokens = word_tokenize(sentence, engine="newmm")
  tokens_join = " ".join(tokens).replace("  ", "")
  batch["sentences"] = tokens_join

  return batch

In [ ]:
from lextoplus import LexToPlus

lexto = LexToPlus()
lexto.tokenize('')

from pythainlp.tokenize import word_tokenize

def th_words_tokenize(batch):
  sentence = batch["cleanSentence"]
  tokens = word_tokenize(sentence, engine="newmm")
  tokens_join = " ".join(tokens).replace("  ", "")
  batch["sentences"] = tokens_join

  return batch

In [ ]:
train_ds = train_ds.map(th_words_tokenize, remove_columns=["cleanSentence"], num_proc=4)
validation_ds = validation_ds.map(th_words_tokenize, remove_columns=["cleanSentence"], num_proc=4)
test_ds = test_ds.map(th_words_tokenize, remove_columns=["cleanSentence"], num_proc=4)

Map (num_proc=4):   0%|          | 0/2917 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1922 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2188 [00:00<?, ? examples/s]

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["labels"] = processor.tokenizer(batch["sentences"]).input_ids

    return batch

In [ ]:
train_ds = train_ds.map(prepare_dataset, remove_columns=train_ds.column_names, num_proc=4)
validation_ds = validation_ds.map(prepare_dataset, remove_columns=validation_ds.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/2917 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1922 [00:00<?, ? examples/s]

In [ ]:
# Save Data for training to local
train_ds.save_to_disk("data/train") # สำหรับฝึกสอน โดยชุดข้อมูลจะเก็บแค่ค่า input_values และ labels
validation_ds.save_to_disk("data/validation") # สำหรับฝึกสอน โดยชุดข้อมูลจะเก็บแค่ค่า input_values และ labels
test_ds.save_to_disk("data/test") # สำหรับทดสอบ โดยชุดข้อมูลจะเก็บข้อมูลไฟล์เสียง array, sample rate, และประโยคต้นฉบับที่คลีนแล้วไว้

Saving the dataset (0/2 shards):   0%|          | 0/2917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1922 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2188 [00:00<?, ? examples/s]

# **Prepared for Trainer**

1.   ฟังก์ชัน DataCollator ในที่นี้สร้างขึ้นสำหรับ Padding ข้อมูลที่ส่งเข้าสู่โมเดล
2.   สร้างฟังก์ชันสำหรับประเมินผล โดยใช้เมทริกซ์คือ WER และยิ่งค่าลดลงเข้าใกล้ 0 แสดงว่ามีผลลัพธ์ที่ดีขึ้น



In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        labels_batch = self.processor.tokenizer.pad(
            label_features,
            padding=self.padding,
            max_length=self.max_length_labels,
            pad_to_multiple_of=self.pad_to_multiple_of_labels,
            return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import evaluate
import numpy as np

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# **Call Pretrained Model and Fine Tune it**

โหลด Pretrained Model ของ Facebook เก็บไว้บน Hugging Face Hub โดยเราจะโหลดตัว Wav2Vec 2.0 Cross-Lingual Speech Recognition ที่ถูกเทรนไว้บน 53 ภาษามาใช้งาน ซึ่งสามารถที่จะทดลองกำหนด Hyperparameter ได้ แต่สิ่งสำคัญคือเราจะต้องกำหนด pad_token_id และ vocab_size ตาม Tokenizer ที่สร้างขึ้น

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

ส่วนนี้เราไม่ต้องการ Fine Tune ตัว Feature Encoder ที่ถูกเทรนมาแล้วของโมเดลจึงทำการ Freeze ไว้แบบเดิม และเรียกใช้งาน gradient_checkpointing_enable เพื่อประหยัดการใช้ Memmory บน GPU

In [ ]:
model.freeze_feature_encoder()
model.gradient_checkpointing_enable()

ในขั้นตอนการเทรนโมเดล เราจะเรียกใช้ Trainer API ของ Hugging Face ซึ่งสามารถตั้งค่าได้ง่าย สะดวกต่อการทดลอง ดูผล และจัดเก็บโมเดลไว้เป็น Checkpoints <br />
<a href="https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments"> พารามิเตอร์ต่าง ๆ สามารถศึกษาได้ที่นี่ </a>

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-xlsr53-TH-cmv-ckp1/", # โฟลเดอร์ที่ต้องการเก็บโมเดลที่ Fine Tuning แล้ว (Checkpoints)
  group_by_length=True, # กรุ๊ปข้อมูล Input ตามขนาด
  per_device_train_batch_size=16, # กำหนด batch_size
  gradient_accumulation_steps=2, # การคำนวณ Gradient
  evaluation_strategy="steps", # กำหนดว่าจะวัดผลเมื่อใด ในที่นี้คือกำหนดตาม Steps
  num_train_epochs=8, # จำนวน epoch
  fp16=True, # Half Precision (torch.float32 --> torch.float16)
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500, # จำนวน Steps ก่อนที่จะเริ่มปรับ Learning Rate
  save_total_limit=2, # จำนวน Checkpoints ที่ต้องการเก็บ ในที่นี้คือเก็บทุก ๆ 2 โมเดลล่าสุด
)

ใส่ฟังก์ชันทั้งหมดลงใน Trainer และทำการเทรน (Fine Tuning) ตัวโมเดล

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2917
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 728
  Number of trainable parameters = 311299269


Step,Training Loss,Validation Loss,Wer
100,6.054000,4.966051,1.000000
200,3.642300,3.659085,1.000000
300,3.598300,3.588786,1.000000
400,3.535700,3.542163,1.000000
500,3.529800,3.481983,1.000000
600,2.385700,2.050508,0.987996
700,1.336300,1.194516,0.841423


***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
Saving model checkpoint to wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-100
Configuration saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-100/config.json
Model weights saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-100/pytorch_model.bin
Feature extractor saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
Saving model checkpoint to wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-200
Configuration saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-200/config.json
Model weights saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-200/pytorch_model.bin
Feature extractor saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
Saving model checkpoint to wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-300
Configuration saved in wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-300/

TrainOutput(global_step=728, training_loss=4.958802777332264, metrics={'train_runtime': 3225.7252, 'train_samples_per_second': 7.234, 'train_steps_per_second': 0.226, 'total_flos': 3.081568169322409e+18, 'train_loss': 4.958802777332264, 'epoch': 7.99})

# **Inference**

1.   โหลด Tokenizer จาก vocab.json ในโฟลเดอร์ Processor ที่เซฟไว้ข้างต้น
2.   โหลด Feature Extractor
3.   แพค Tokenizer และ Feature Extractor ไว้ใน Processor
4.   โหลดโมเดล Wav2Vec2 จาก Checkpoints ล่าสุด
5.   ทดสอบถอดเสียงพูดภาษาไทย

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/WAV2VEC2/NT/vocab.json",
                                 unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

ModuleNotFoundError: ignored

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/wav2vec2/NT/",
                                       attention_dropout=0.1,
                                       hidden_dropout=0.1,
                                       feat_proj_dropout=0.0,
                                       mask_time_prob=0.05,
                                       layerdrop=0.1,
                                       ctc_loss_reduction="mean",
                                       pad_token_id=processor.tokenizer.pad_token_id,
                                       vocab_size=len(processor.tokenizer))

device = 'cuda' if torch.cuda.is_available else 'cpu'
model.to(device)

ModuleNotFoundError: ignored

In [ ]:
import torch
import numpy as np

id = 0
wav_array = test_ds[id]["audio"]["array"]
sr = test_ds[id]["audio"]["sampling_rate"]
reference = test_ds[id]["sentences"]

input_values = torch.tensor(processor(wav_array, sampling_rate=sr).input_values)

with torch.no_grad():
  logits = model(input_values.to(device)).logits

In [ ]:
import IPython.display as ipd
import numpy as np
import random

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", reference)

ipd.Audio(data=wav_array, autoplay=True, rate=sr)

Prediction: ['เรา เริมตนดวย วิทีนนี้']
Reference: เรา เริ่มต้น ด้วย วิธี นี้


# **Resume Training**
หากต้องการเทรนโมเดลต่อจากเดิม สามารถทำได้ตามขั้นตอนนี้

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/finetune-wav2vec2/

/content/drive/MyDrive/finetune-wav2vec2


In [ ]:
# Need to install every time after terminate runtine
!pip install transformers
!pip install datasets
!pip install librosa
!pip install jiwer
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.7 MB/s eta 0:00:

In [ ]:
from datasets import load_from_disk

train_ds = load_from_disk("data/train/")
validation_ds = load_from_disk("data/validation/")
test_ds = load_from_disk("data/test/")

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/finetune-wav2vec2/wav2vec2-xlsr53-TH-cmv-processor/vocab.json",
                                 unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/finetune-wav2vec2/wav2vec2-xlsr53-TH-cmv-ckp1/checkpoint-700/",
                                       attention_dropout=0.1,
                                       hidden_dropout=0.1,
                                       feat_proj_dropout=0.0,
                                       mask_time_prob=0.05,
                                       layerdrop=0.1,
                                       ctc_loss_reduction="mean",
                                       pad_token_id=processor.tokenizer.pad_token_id,
                                       vocab_size=len(processor.tokenizer))

In [ ]:
model.freeze_feature_encoder()
model.gradient_checkpointing_enable()

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        labels_batch = self.processor.tokenizer.pad(
            label_features,
            padding=self.padding,
            max_length=self.max_length_labels,
            pad_to_multiple_of=self.pad_to_multiple_of_labels,
            return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import evaluate
import numpy as np

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-xlsr53-TH-cmv-ckp2/", # Change output_dir when resume training
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=8,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2917
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 728
  Number of trainable parameters = 311299269


Step,Training Loss,Validation Loss,Wer
100,1.244600,1.110248,0.816747
200,1.025300,0.982060,0.773249
300,0.860300,0.879671,0.714635
400,0.757300,0.832022,0.691219
500,0.710700,0.826554,0.670693
600,0.539200,0.772061,0.635717
700,0.421200,0.761336,0.622971


***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
Saving model checkpoint to wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-100
Configuration saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-100/config.json
Model weights saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-100/pytorch_model.bin
Feature extractor saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
Saving model checkpoint to wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-200
Configuration saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-200/config.json
Model weights saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-200/pytorch_model.bin
Feature extractor saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
Saving model checkpoint to wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-300
Configuration saved in wav2vec2-xlsr53-TH-cmv-ckp2/checkpoint-300/

TrainOutput(global_step=728, training_loss=0.8435883973980998, metrics={'train_runtime': 3321.6416, 'train_samples_per_second': 7.025, 'train_steps_per_second': 0.219, 'total_flos': 3.081568169322409e+18, 'train_loss': 0.8435883973980998, 'epoch': 7.99})

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import numpy as np

id = 0
wav_array = test_ds[id]["audio"]["array"]
sr = test_ds[id]["audio"]["sampling_rate"]
reference = test_ds[id]["sentences"]

input_values = torch.tensor(processor(wav_array, sampling_rate=sr).input_values)

with torch.no_grad():
  logits = model(input_values.to(device)).logits

/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
import IPython.display as ipd
import numpy as np
import random

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", reference)

ipd.Audio(data=wav_array, autoplay=True, rate=sr)

Prediction: ['เรา เริม ต้น ด้วย วิธียนี้']
Reference: เรา เริ่มต้น ด้วย วิธี นี้
